In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
from graph_utils import graph_gen
from algorithms import Falsone17DualDecomp

# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

# from algorithms import B_DPP, Falsone, C_SP_SG, DPD_TV

num_node = 10
num_edge = 25

graph_gen(num_node, num_edge, B=1)

net_dir = f'data/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)

Graph with 10 nodes and 25 edges
(1, 10, 10)


C:\Users\xtq\AppData\Roaming\Python\Python311\site-packages\networkx\drawing\nx_pylab.py:315: UserWarning: 

The arrowsize keyword argument is not applicable when drawing edges
with LineCollection.

To make this warning go away, either specify `arrows=True` to
force FancyArrowPatches or use the default values.
Note that using FancyArrowPatches may be slow for large graphs.

  draw_networkx_edges(G, pos, arrows=arrows, **edge_kwds)


In [2]:
from problem import VNResAlloc

prob = VNResAlloc(num_node, d=5)
prob.gen()
# prob.load()


generating a VN resource allocation problem
generated problem saved in data/problem/VNResAlloc/N10_d5


In [3]:
MAX_ITER = 500

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# obj_err_list = []
# cons_val_list = []
# for algo_idx in range(len(algo_list)):
    # algo = algo_list[algo_idx]

alg = Falsone17DualDecomp(prob, network, beta=4.4)
# logging.info(f'{labels[algo_idx]} is running:')
obj_err_log = []
cons_vio_log = []
obj_err, cons_vio = alg.compute_metrics(opt_val=prob.opt_val)
obj_err_log.append(obj_err)
cons_vio_log.append(cons_vio)
for t in range(MAX_ITER):
    alg.step()
    obj_err, cons_vio = alg.compute_metrics(opt_val=prob.opt_val)
    obj_err_log.append(obj_err)
    cons_vio_log.append(cons_vio)
    logging.info(f'iter {alg.iter_num}, obj err: {obj_err:.2e}, cons vio: {cons_vio}')
# obj_err_list.append(obj_err_log)
# cons_val_list.append(cons_vio_log)


iter 1, obj err: 2.02e+00, cons vio: 29.48925912105524
iter 2, obj err: 2.28e+00, cons vio: 19.772473345829553
iter 3, obj err: 2.11e+00, cons vio: 13.342760508033095
iter 4, obj err: 1.92e+00, cons vio: 9.988634448138109
iter 5, obj err: 1.71e+00, cons vio: 7.306033323353158
iter 6, obj err: 1.55e+00, cons vio: 5.688337588897918
iter 7, obj err: 1.40e+00, cons vio: 4.532967417051431
iter 8, obj err: 1.29e+00, cons vio: 3.8840140135905874
iter 9, obj err: 1.19e+00, cons vio: 3.3616033531280456
iter 10, obj err: 1.11e+00, cons vio: 2.94617987007273
iter 11, obj err: 1.04e+00, cons vio: 2.5989079894330684
iter 12, obj err: 9.79e-01, cons vio: 2.307190910603138
iter 13, obj err: 9.27e-01, cons vio: 2.058075524227932
iter 14, obj err: 8.80e-01, cons vio: 1.8430115808480572
iter 15, obj err: 8.39e-01, cons vio: 1.6554791907555413
iter 16, obj err: 8.03e-01, cons vio: 1.5077230681571385
iter 17, obj err: 7.70e-01, cons vio: 1.4106162207393962
iter 18, obj err: 7.41e-01, cons vio: 1.323972166

In [4]:
np.savetxt(f'{log_dir}/obj_err_{alg.name}.txt', obj_err_log, delimiter=',')

# print(cons_vio_log)
np.savetxt(f'{log_dir}/cons_vio_{alg.name}.txt', cons_vio_log, delimiter=',')

In [5]:
from plot_utils import MyFigure

cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $t$', 
                            ylabel=r'$\sum_{i=1}^N g_i(\bar{x}_{i,t})$',
                            yscale='symlog')
cons_vio_figure.add_line("Falsone 17 DD", cons_vio_log)
cons_vio_figure.paint(MAX_ITER=MAX_ITER)

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $t$', 
                            ylabel=r'$\left\vert\sum_{i=1}^N f_i(\bar{x}_{i,t}) - \sum_{i=1}^N f_i(x_i^\star)\right\vert$')
obj_err_figure.add_line("Falsone 17 DD", obj_err_log)
obj_err_figure.paint(MAX_ITER=MAX_ITER)